```bash
# ================================
# Kali Linux — Command Cheat Sheet
# ================================

# 1) System updates & cleanup
sudo apt update && sudo apt full-upgrade -y
sudo apt autoremove --purge -y
sudo apt clean

# 2) Users, groups, permissions (least privilege)
sudo adduser <username>
sudo usermod -aG sudo <username>
sudo passwd -l root                       # (optional) disable local root login
sudo chown <user>:<group> <path>          # change owner
sudo chmod <perm> <path>                  # change permissions (e.g., 600, 700, 750)
chmod -R 700 ~/.ssh && chmod 600 ~/.ssh/* # secure permissions for SSH keys
passwd # change current user's password
sudo useradd test # add a new user "test"

# 3) Two-Factor Authentication (Google Authenticator)
sudo apt install -y libpam-google-authenticator
google-authenticator                      # interactive setup for current user
echo "auth required pam_google_authenticator.so nullok" | sudo tee -a /etc/pam.d/sshd
sudo sed -i 's/^#\?ChallengeResponseAuthentication.*/ChallengeResponseAuthentication yes/' /etc/ssh/sshd_config
sudo systemctl restart ssh

# 4) Service hardening (disable what you don't need)
systemctl list-unit-files --type=service --state=enabled
sudo systemctl disable --now <service_name>
sudo systemctl mask <service_name>        # prevents even manual startup

# 5) Firewall (UFW)
sudo apt install -y ufw
sudo ufw default deny incoming
sudo ufw default allow outgoing
sudo ufw allow 22/tcp                     # or the SSH port you will use (see SSH section)
# More restrictive examples:
# sudo ufw allow from <IP>/32 to any port 2222 proto tcp
sudo ufw enable
sudo ufw status verbose # 

# 6) SSH hardening
sudo apt install -y openssh-server
sudo sed -i 's/^#\?PermitRootLogin.*/PermitRootLogin no/' /etc/ssh/sshd_config
sudo sed -i 's/^#\?PasswordAuthentication.*/PasswordAuthentication no/' /etc/ssh/sshd_config
sudo sed -i 's/^#\?Port .*/Port 2222/' /etc/ssh/sshd_config   # choose a non-standard port
# (optional) limit users/IPs:
# echo -e "AllowUsers <user1> <user2>\n" | sudo tee -a /etc/ssh/sshd_config
sudo systemctl restart ssh

# 7) VPN (OpenVPN / WireGuard)
sudo apt install -y openvpn
sudo openvpn --config /path/to/profile.ovpn
sudo apt install -y wireguard
sudo wg-quick up wg0                       # requires /etc/wireguard/wg0.conf

# 8) Backup (rsync + Timeshift)
# Example rsync (incremental backup with ACL/xattr preservation):
sudo rsync -aAXHv --delete --numeric-ids --info=progress2 \
  /etc /home /var/backups /usr/local \
  /mnt/backup/kali-$(date +%F)/
# Timeshift (system snapshots — useful on desktop installations):
sudo apt install -y timeshift
sudo timeshift --create --comments "pre-change" --tags D

# 9) Data encryption
sudo apt install -y veracrypt
# GnuPG (symmetric/asymmetric encryption):
gpg --gen-key
gpg -c <file>                              # symmetric encryption -> <file>.gpg
gpg -e -r "<Your GPG Name or Email>" <file>
gpg -d <file>.gpg > <file>.dec

# 10) IDPS (Intrusion Detection & Prevention)
sudo apt install -y snort
sudo snort -T -c /etc/snort/snort.conf     # test configuration
sudo apt install -y suricata
sudo suricata -T -c /etc/suricata/suricata.yaml
sudo systemctl enable --now suricata
# (optional) OSSEC / Wazuh (if available in configured repos):
# sudo apt install -y ossec-hids
# sudo systemctl enable --now ossec

# 11) Quick monitoring (SSH logs)
sudo journalctl -u ssh --since "today"
sudo tail -n 100 /var/log/auth.log

# 12) Scheduling (cron example for weekly rsync backup)
# crontab -e
# 0 3 * * 1 sudo rsync -aAXHv --delete /etc /home /var/backups /usr/local /mnt/backup/kali-$(date +\%F)/

# ===========================================
# What are SUDO and APT?
# ===========================================

# SUDO (Super User DO)
# --------------------
# - A program that allows you to run commands with superuser (root) privileges
# - Located at: /usr/bin/sudo
# - Essential for system administration tasks
# - Provides security by controlling privileged access

# Examples:
sudo apt update           # Run 'apt update' as administrator
sudo nano /etc/hosts      # Edit protected system files
sudo reboot              # Restart the system

# WARNING: Use sudo only when necessary!
# Commands with sudo can modify the entire system.

# APT (Advanced Package Tool)
# ---------------------------
# - A package management system for Debian/Ubuntu/Kali Linux
# - Located at: /usr/bin/apt
# - Handles software installation, removal, and updates
# - Works with software repositories

# Examples:
sudo apt update          # Refresh available package list
sudo apt upgrade         # Upgrade installed packages
sudo apt install firefox # Install software
sudo apt remove firefox  # Remove software
apt search python        # Search for packages
apt list --installed    # List installed packages

# Common combinations:
sudo apt update && sudo apt upgrade -y    # Update everything
sudo apt autoremove --purge -y           # Remove unnecessary packages

```
